In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from pathlib import Path

notebook_path = "/u/skarmakar1/version_check/llm_steering-main/sk"
sys.path.append(notebook_path)

In [3]:
import torch
import numpy as np

from inversion_utils import *
import pickle
from sklearn.model_selection import train_test_split

In [4]:
SEED = 0

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)

torch.backends.cudnn.benchmark = True 
torch.backends.cuda.matmul.allow_tf32 = True

LLM = namedtuple('LLM', ['language_model', 'tokenizer', 'processor', 'name', 'model_type'])

In [5]:
model_type = 'llama'
# model_type = 'qwen'

# MODEL_VERSION = '3'
MODEL_VERSION = '3.1'
# MODEL_VERSION = '3.3'

MODEL_SIZE = '8B'
# MODEL_SIZE = '70B'

llm = select_llm(model_type, MODEL_VERSION=MODEL_VERSION, MODEL_SIZE=MODEL_SIZE)

Loading meta-llama/Meta-Llama-3.1-8B-Instruct


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Neural ODE

In [6]:
# pip install torchdiffeq
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchdiffeq import odeint_adjoint as odeint  # O(1)-memory adjoint

from copy import deepcopy
import logging

In [7]:
def setup_logger(log_file='training.log'):
    """
    Setup logger to write to file and console
    """
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()  # Also print to console
        ]
    )

In [8]:
# 1) Neural vector field f_theta(t, x): R^4096 -> R^4096, implements dx/dt
class VectorField(nn.Module):
    def __init__(self, dim=4096, hidden=1024, use_time=True, use_norm=True):
        super().__init__()
        in_dim = dim + (1 if use_time else 0)
        self.use_time = use_time
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.Tanh(),
            # nn.Linear(hidden, hidden),
            # nn.Tanh(),
            nn.Linear(hidden, dim)
        )
        self.use_norm = use_norm
        self.norm = nn.LayerNorm(dim)

    def forward(self, t, x):
        if self.use_time:
            t_feat = torch.full((x.shape[0], 1), float(t), device=x.device, dtype=x.dtype)
            z = torch.cat([x, t_feat], dim=1)
        else:
            z = x
        dx = self.net(z)

        if self.use_norm:
            return self.norm(dx)
        else:
            return dx

# 2) ODE block that integrates from t0=0 to t1=1
class NeuralODE(nn.Module):
    def __init__(self, vf, method='dopri5', rtol=1e-5, atol=1e-5):
        super().__init__()
        self.vf = vf
        self.method = method
        self.rtol = rtol
        self.atol = atol

    def forward(self, y0, limits=[0.0, 1.0]):
        t = torch.tensor(limits, device=y0.device, dtype=y0.dtype)
        y = odeint(self.vf, y0, t, method=self.method, rtol=self.rtol, atol=self.atol)
        return y[-1]

# 3) Training with optional validation; prints train and validation loss each epoch
def train_neural_ode(
    X, Y, X_val=None, Y_val=None, *,
    epochs=50, batch_size=32, lr=1e-3, weight_decay=1e-3, hidden=1024, use_time=True, use_norm=True, loss_type="mse", device=None, log_file="ode_logs/training.log",
):

    setup_logger(log_file)

    assert X.ndim == 2 and Y.ndim == 2 and X.shape == Y.shape and X.shape[1] == 4096
    if X_val is not None and Y_val is not None:
        assert X_val.ndim == 2 and Y_val.ndim == 2 and X_val.shape == Y_val.shape and X_val.shape[1] == 4096
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")

    logging.info("*"*50)
    logging.info(f'Starting training on device: {device}')
    logging.info(f'Number of epochs: {epochs}, Learning rate: {lr}')

    ds = TensorDataset(X, Y)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=True, drop_last=False)

    val_dl = None
    if X_val is not None and Y_val is not None:
        val_ds = TensorDataset(X_val, Y_val)
        val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False, drop_last=False)

    # vf = VectorField(dim=4096, hidden=hidden, use_time=True, use_norm=True).to(device)
    vf = VectorField(dim=4096, hidden=hidden, use_time=use_time, use_norm=use_norm).to(device)
    
    model = NeuralODE(vf, method='dopri5', rtol=1e-5, atol=1e-5).to(device)

    opt = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    if loss_type == "mse":
        loss_fn = nn.MSELoss()
    
    elif loss_type == "cosine":
        loss_fn = nn.CosineEmbeddingLoss(margin=0.0, reduction='mean')

    best_val_loss = float('inf')
    best_model_state = None

    for epoch in range(1, epochs + 1):
        # Train
        model.train()
        running = 0.0
        for xb, yb in dl:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            pred = model(xb)

            if loss_type == "mse":
                loss = loss_fn(pred, yb)
            
            elif loss_type == "cosine":
                target_labels = torch.ones(xb.size(0), device=xb.device)  # [B], all +1
                loss = loss_fn(pred, yb, target_labels)

            loss.backward()
            opt.step()
            running += loss.item() * xb.size(0)
        train_loss = running / len(ds)

        # Validate (minimal additions)
        if val_dl is not None:
            model.eval()  # set eval mode for layers like dropout/BN
            vrunning = 0.0
            with torch.no_grad():  # disable autograd for speed/memory
                for xb, yb in val_dl:
                    xb, yb = xb.to(device), yb.to(device)
                    vpred = model(xb)

                    if loss_type == "mse":
                        vloss = loss_fn(vpred, yb)
                    
                    elif loss_type == "cosine":
                        vlabels = torch.ones(xb.size(0), device=xb.device)
                        vloss = loss_fn(vpred, yb, vlabels)

                    vrunning += vloss.item() * xb.size(0)
            val_loss = vrunning / len(val_dl.dataset)
            # print(f"epoch {epoch}: train_loss={train_loss:.6f}  val_loss={val_loss:.6f}")
            logging.info(f"epoch {epoch}: train_loss={train_loss:.6f}  val_loss={val_loss:.6f}")

            # Save best model state
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = deepcopy(model.state_dict())

        else:
            # print(f"epoch {epoch}: train_loss={train_loss:.6f}")
            logging.info(f"epoch {epoch}: train_loss={train_loss:.6f}")

    # Load best model state if validation was used
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        logging.info(f'Best model loaded with validation loss: {best_val_loss:.6f}')
    
    return model

# # 4) Example usage (replace with your actual tensors)
# if __name__ == "__main__":
#     N, Nv = 512, 128
#     X = torch.randn(N, 4096)
#     W = torch.randn(4096, 4096) / 4096**0.5
#     Y = X @ W.T

#     X_val = torch.randn(Nv, 4096)
#     Y_val = X_val @ W.T

#     model = train_neural_ode(X, Y, X_val=X_val, Y_val=Y_val,
#                              epochs=10, batch_size=16, lr=1e-3, hidden=1024)

Training

In [9]:
with open("../data/moods/all_antonym_pairs.pkl", 'rb') as file:
    all_e = pickle.load(file)

In [10]:
print("Total data:", len(all_e))
print(all_e[:5])

train_data_t, test_data = train_test_split(all_e, test_size=0.1, random_state=SEED)

print("Training data normal:", len(train_data_t))
print(train_data_t[:5])

swap_train_data = [(b, a) for a, b in train_data_t]
print("Training data swapped:", len(swap_train_data))
print(swap_train_data[:5])

train_data = train_data_t + swap_train_data
print("Training data:", len(train_data))
print(train_data[:5])

print("Testing data:", len(test_data))
print(test_data[:5])

Total data: 239
[('optimistic', 'pessimistic'), ('enthusiastic', 'apathetic'), ('energetic', 'lethargic'), ('tense', 'relaxed'), ('confident', 'diffident')]
Training data normal: 215
[('delighted', 'disappointed'), ('sympathetic', 'unsympathetic'), ('burdened', 'unburdened'), ('lighthearted', 'somber'), ('insincere', 'sincere')]
Training data swapped: 215
[('disappointed', 'delighted'), ('unsympathetic', 'sympathetic'), ('unburdened', 'burdened'), ('somber', 'lighthearted'), ('sincere', 'insincere')]
Training data: 430
[('delighted', 'disappointed'), ('sympathetic', 'unsympathetic'), ('burdened', 'unburdened'), ('lighthearted', 'somber'), ('insincere', 'sincere')]
Testing data: 24
[('surprised', 'unsurprised'), ('hasty', 'careful'), ('evasive', 'direct'), ('pretentious', 'unpretentious'), ('confrontational', 'diplomatic')]


In [11]:
print(test_data)

[('surprised', 'unsurprised'), ('hasty', 'careful'), ('evasive', 'direct'), ('pretentious', 'unpretentious'), ('confrontational', 'diplomatic'), ('contemptible', 'respectable'), ('aggressive', 'docile'), ('alarmed', 'unalarmed'), ('spirited', 'dispirited'), ('fervent', 'lukewarm'), ('domineering', 'meek'), ('revolted', 'charmed'), ('terrified', 'dauntless'), ('suspicious', 'trusting'), ('bothered', 'unbothered'), ('stressed', 'rested'), ('bewildered', 'clear-headed'), ('deceitful', 'honest'), ('baffled', 'informed'), ('cynical', 'naïve'), ('certain', 'uncertain'), ('distant', 'close'), ('aloof', 'warm'), ('bold', 'shy')]


In [12]:
X_train, Y_train = read_tuples(llm, train_data, path='../directions_moods_plus_llama/')

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

D

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components

/u/skarmakar1/miniconda3/envs/neuinv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components

In [13]:
print(X_train[-1].shape)
print(Y_train[-1].shape)

torch.Size([430, 4096])
torch.Size([430, 4096])


In [14]:
X_test, Y_test = read_tuples(llm, test_data, path='../directions_moods_plus_llama/')

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

D

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components

In [15]:
print(X_test[-1].shape)
print(Y_test[-1].shape)

torch.Size([24, 4096])
torch.Size([24, 4096])


In [16]:
X_mean = {layer: X_train[layer].mean(axis=0, keepdim=True) for layer in X_train}
X_std = {layer: X_train[layer].std(axis=0, keepdim=True) + 1e-8 for layer in X_train}

Y_mean = {layer: Y_train[layer].mean(axis=0, keepdim=True) for layer in Y_train}
Y_std = {layer: Y_train[layer].std(axis=0, keepdim=True) + 1e-8 for layer in Y_train}

In [17]:
print(X_mean[-1].shape)
print(X_std[-1].shape)

torch.Size([1, 4096])
torch.Size([1, 4096])


In [18]:
X_train_normalized = {layer: (X_train[layer] - X_mean[layer]) / X_std[layer] for layer in X_train}
X_test_normalized = {layer: (X_test[layer] - X_mean[layer]) / X_std[layer] for layer in X_test}

Y_train_normalized = {layer: (Y_train[layer] - Y_mean[layer]) / Y_std[layer] for layer in Y_train}
Y_test_normalized = {layer: (Y_test[layer] - Y_mean[layer]) / Y_std[layer] for layer in Y_test}

In [19]:
print(X_train_normalized[-1])
print(X_train_normalized[-1].shape)
print(Y_train_normalized[-1].shape)

tensor([[ 0.9776, -2.9618,  0.4722,  ..., -1.8523, -0.4136,  1.7110],
        [ 0.4740,  0.2497, -0.2247,  ...,  0.6161, -0.3363,  1.5035],
        [-0.3800,  0.4173,  0.3802,  ...,  0.9986,  0.2623,  0.0422],
        ...,
        [ 0.3103, -1.5643,  0.4502,  ...,  1.5980, -1.3296,  0.9085],
        [-1.1201, -3.0029,  0.4976,  ...,  1.2147, -0.7244, -0.6848],
        [-1.5209, -0.0449, -2.5569,  ...,  0.1781, -1.1896, -1.3363]],
       device='cuda:0')
torch.Size([430, 4096])
torch.Size([430, 4096])


In [20]:
all_node = {}

# # works decent
# folder = "try2"
# use_time = False
# use_norm = False
# loss_type = "mse"
# hidden = 1024

folder = "try3"
use_time = True
use_norm = False
loss_type = "mse"
hidden = 1024

for layer in X_train_normalized:
    print(f"Layer: {layer}")
    X = X_train_normalized[layer]
    Y = Y_train_normalized[layer]

    X_val = X_test_normalized[layer]
    Y_val = Y_test_normalized[layer]

    # model = train_neural_ode(X, Y, X_val=X_val, Y_val=Y_val, epochs=10, batch_size=16, lr=1e-3, hidden=1024)
    # model = train_neural_ode(X, Y, X_val=X_val, Y_val=Y_val, epochs=20, batch_size=32, lr=1e-3, weight_decay=1e-3, hidden=256, log_file="ode_logs/training_try1.log") # decent
    # model = train_neural_ode(X, Y, X_val=X_val, Y_val=Y_val, epochs=50, batch_size=32, lr=1e-3, weight_decay=1e-4, hidden=256, log_file="ode_logs/training_try2.log")
    model = train_neural_ode(X, Y, 
        X_val=X_val, 
        Y_val=Y_val, 
        epochs=20, 
        batch_size=32, 
        lr=1e-3, 
        # weight_decay=1e-4, 
        weight_decay=5e-4, 
        hidden=hidden, 
        use_time=use_time, 
        use_norm=use_norm, 
        loss_type=loss_type, 
        log_file=f"ode_logs/training_{folder}.log")

    all_node[layer] = model

    # break

    # with open(f'ode_ckpt/{folder}/ode_layer{layer}.pickle', 'wb') as file:
    #     pickle.dump(model, file)
    # del model

2025-11-03 19:33:34,082 - **************************************************
2025-11-03 19:33:34,082 - Starting training on device: cuda
2025-11-03 19:33:34,084 - Number of epochs: 20, Learning rate: 0.001


Layer: -1


2025-11-03 19:33:35,590 - epoch 1: train_loss=1.737672  val_loss=1.564946
2025-11-03 19:33:37,293 - epoch 2: train_loss=1.260569  val_loss=1.436714
2025-11-03 19:33:38,719 - epoch 3: train_loss=1.024383  val_loss=1.363587
2025-11-03 19:33:39,877 - epoch 4: train_loss=0.867549  val_loss=1.358712
2025-11-03 19:33:40,923 - epoch 5: train_loss=0.795773  val_loss=1.332808
2025-11-03 19:33:42,091 - epoch 6: train_loss=0.735601  val_loss=1.347492
2025-11-03 19:33:43,125 - epoch 7: train_loss=0.682665  val_loss=1.310263
2025-11-03 19:33:44,080 - epoch 8: train_loss=0.645269  val_loss=1.341814
2025-11-03 19:33:45,229 - epoch 9: train_loss=0.615969  val_loss=1.313693
2025-11-03 19:33:46,258 - epoch 10: train_loss=0.598267  val_loss=1.337135
2025-11-03 19:33:47,386 - epoch 11: train_loss=0.580076  val_loss=1.351759
2025-11-03 19:33:48,438 - epoch 12: train_loss=0.556932  val_loss=1.315632
2025-11-03 19:33:49,600 - epoch 13: train_loss=0.537786  val_loss=1.319954
2025-11-03 19:33:50,660 - epoch 14

Layer: -2


2025-11-03 19:33:58,655 - epoch 1: train_loss=1.735993  val_loss=1.568279
2025-11-03 19:34:00,524 - epoch 2: train_loss=1.259805  val_loss=1.441482
2025-11-03 19:34:02,053 - epoch 3: train_loss=1.029095  val_loss=1.388034
2025-11-03 19:34:03,122 - epoch 4: train_loss=0.872747  val_loss=1.346169
2025-11-03 19:34:04,213 - epoch 5: train_loss=0.793822  val_loss=1.361541
2025-11-03 19:34:05,231 - epoch 6: train_loss=0.735291  val_loss=1.331742
2025-11-03 19:34:06,260 - epoch 7: train_loss=0.686036  val_loss=1.359763
2025-11-03 19:34:07,244 - epoch 8: train_loss=0.650033  val_loss=1.332972
2025-11-03 19:34:08,268 - epoch 9: train_loss=0.628427  val_loss=1.312931
2025-11-03 19:34:09,413 - epoch 10: train_loss=0.595943  val_loss=1.304843
2025-11-03 19:34:10,517 - epoch 11: train_loss=0.572328  val_loss=1.303368
2025-11-03 19:34:11,683 - epoch 12: train_loss=0.556196  val_loss=1.351605
2025-11-03 19:34:12,752 - epoch 13: train_loss=0.548322  val_loss=1.320524
2025-11-03 19:34:13,905 - epoch 14

...

2025-11-03 19:44:54,916 - epoch 1: train_loss=1.545604  val_loss=1.277278
2025-11-03 19:44:57,495 - epoch 2: train_loss=1.248323  val_loss=1.216563
2025-11-03 19:45:00,197 - epoch 3: train_loss=1.137596  val_loss=1.236909
2025-11-03 19:45:02,840 - epoch 4: train_loss=1.039083  val_loss=1.162754
2025-11-03 19:45:04,909 - epoch 5: train_loss=1.040860  val_loss=1.190445
2025-11-03 19:45:07,205 - epoch 6: train_loss=0.981787  val_loss=1.117852
2025-11-03 19:45:09,311 - epoch 7: train_loss=0.923467  val_loss=1.253896
2025-11-03 19:45:11,358 - epoch 8: train_loss=0.898712  val_loss=1.328923
2025-11-03 19:45:13,330 - epoch 9: train_loss=0.876389  val_loss=1.259985
2025-11-03 19:45:15,136 - epoch 10: train_loss=0.807735  val_loss=1.329611
2025-11-03 19:45:16,909 - epoch 11: train_loss=0.803738  val_loss=1.284510
2025-11-03 19:45:18,576 - epoch 12: train_loss=0.747823  val_loss=1.351710
2025-11-03 19:45:20,443 - epoch 13: train_loss=0.749828  val_loss=1.248279
2025-11-03 19:45:21,977 - epoch 14

Layer: -30


2025-11-03 19:45:31,957 - epoch 1: train_loss=1.646305  val_loss=1.518215
2025-11-03 19:45:34,374 - epoch 2: train_loss=1.257656  val_loss=1.448983
2025-11-03 19:45:36,972 - epoch 3: train_loss=1.123174  val_loss=1.426241
2025-11-03 19:45:39,454 - epoch 4: train_loss=1.028891  val_loss=1.416514
2025-11-03 19:45:42,260 - epoch 5: train_loss=1.000370  val_loss=1.403690
2025-11-03 19:45:45,031 - epoch 6: train_loss=0.968085  val_loss=1.411513
2025-11-03 19:45:47,835 - epoch 7: train_loss=0.945730  val_loss=1.399118
2025-11-03 19:45:50,859 - epoch 8: train_loss=0.908845  val_loss=1.441192
2025-11-03 19:45:53,860 - epoch 9: train_loss=0.890283  val_loss=1.413518
2025-11-03 19:45:56,797 - epoch 10: train_loss=0.882396  val_loss=1.404998
2025-11-03 19:45:59,791 - epoch 11: train_loss=0.862590  val_loss=1.399245
2025-11-03 19:46:02,783 - epoch 12: train_loss=0.887003  val_loss=1.428279
2025-11-03 19:46:05,725 - epoch 13: train_loss=0.886621  val_loss=1.419016
2025-11-03 19:46:08,836 - epoch 14

Layer: -31


2025-11-03 19:46:29,649 - epoch 1: train_loss=1.681782  val_loss=1.649151
2025-11-03 19:46:31,400 - epoch 2: train_loss=1.282719  val_loss=1.572407
2025-11-03 19:46:32,974 - epoch 3: train_loss=1.051886  val_loss=1.512264
2025-11-03 19:46:34,080 - epoch 4: train_loss=0.926565  val_loss=1.544564
2025-11-03 19:46:35,340 - epoch 5: train_loss=0.846224  val_loss=1.539580
2025-11-03 19:46:36,256 - epoch 6: train_loss=0.778441  val_loss=1.498365
2025-11-03 19:46:37,256 - epoch 7: train_loss=0.740886  val_loss=1.536372
2025-11-03 19:46:38,151 - epoch 8: train_loss=0.702633  val_loss=1.564588
2025-11-03 19:46:39,125 - epoch 9: train_loss=0.665912  val_loss=1.531218
2025-11-03 19:46:40,043 - epoch 10: train_loss=0.643113  val_loss=1.535196
2025-11-03 19:46:40,996 - epoch 11: train_loss=0.615752  val_loss=1.575781
2025-11-03 19:46:41,921 - epoch 12: train_loss=0.586719  val_loss=1.544629
2025-11-03 19:46:42,823 - epoch 13: train_loss=0.568760  val_loss=1.653616
2025-11-03 19:46:43,791 - epoch 14

In [21]:
all_stats = {
    # 'all_predictors': all_predictors,
    'X_mean': X_mean,
    'X_std': X_std,
    'Y_mean': Y_mean,
    'Y_std': Y_std,
}

# with open(f'ode_ckpt/{folder}/all_stats.pickle', 'wb') as file:
#     pickle.dump(all_stats, file)

Load and run

In [9]:
def ode_predict(all_predictors, X_new, X_mean, X_std, Y_mean, Y_std, limits=[0.0, 1.0], device='cuda'):
    predictions = {}

    for layer in all_predictors:
        model = all_predictors[layer].to(device)
        model.eval()
        
        # Step 1: Normalize input using training statistics
        X_normalized = ((X_new[layer] - X_mean[layer]) / X_std[layer]).to(device)

        # Step 2: Run model inference
        with torch.no_grad():
            # X_tensor = torch.FloatTensor(X_normalized).to(device)
            predictions_normalized = model(X_normalized, limits=limits)
            # predictions_normalized = predictions_normalized.cpu().numpy()
        
        # Step 3: Denormalize output using training statistics
        predictions[layer] = predictions_normalized * Y_std[layer] + Y_mean[layer]
        # predictions[layer] = predictions_normalized
    
    return predictions

In [10]:
folder = "try2"

with open(f"ode_ckpt/{folder}/all_stats.pickle", 'rb') as file:
    all_stats = pickle.load(file)

X_mean = all_stats["X_mean"]
X_std = all_stats["X_std"]
Y_mean = all_stats["Y_mean"]
Y_std = all_stats["Y_std"]

/u/skarmakar1/miniconda3/envs/neuinv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [11]:
all_node = {}

layers = list(range(-1, -llm.language_model.config.num_hidden_layers, -1))

for layer in layers:
    with open(f'ode_ckpt/{folder}/ode_layer{layer}.pickle', 'rb') as file:
        all_node[layer] = pickle.load(file)

Inverse check

In [13]:
coef=0.75
max_tokens=200

prompts = ["There is a stray dog in front of my house, what should I do?",]

controller_1 = load_controller(llm, "aggressive", path='../directions_moods_plus_llama/')
orig_1 = controller_1.directions

controller_2 = load_controller(llm, "docile", path='../directions_moods_plus_llama/')
orig_2 = controller_2.directions

test_concept_vector(controller_1, concept="aggressive", prompts=prompts, coef=coef, max_tokens=max_tokens)
test_concept_vector(controller_2, concept="docile", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

coef = 0.75
# aggresive
# 2 way 1 time
inv_c1 = ode_predict(all_node, controller_1.directions, X_mean, X_std, Y_mean, Y_std)
controller_1.directions = inv_c1
test_concept_vector(controller_1, concept="inverted aggressive ode", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

# 2 way 2 time
# inv_inv_c1 = apply_trans(inv_c1, lrr_matrix_2way)
inv_inv_c1 = ode_predict(all_node, inv_c1, X_mean, X_std, Y_mean, Y_std)
controller_1.directions = inv_inv_c1
test_concept_vector(controller_1, concept="inverted^2 aggressive ode", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

# 2 way 3 time
# inv_inv_c1 = apply_trans(inv_c1, lrr_matrix_2way)
inv_inv_inv_c1 = ode_predict(all_node, inv_inv_c1, X_mean, X_std, Y_mean, Y_std)
controller_1.directions = inv_inv_inv_c1
test_concept_vector(controller_1, concept="inverted^3 aggressive ode", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

# 2 way 4 time
# inv_inv_c1 = apply_trans(inv_c1, lrr_matrix_2way)
inv_inv_inv_inv_c1 = ode_predict(all_node, inv_inv_inv_c1, X_mean, X_std, Y_mean, Y_std)
controller_1.directions = inv_inv_inv_inv_c1
test_concept_vector(controller_1, concept="inverted^4 aggressive ode", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)


Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found

========================== No Control ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

There is a stray dog in front of my house, what should I do?<|eot_id|><|start_head

In [ ]:
# ========================== + inverted aggressive ode Control (normal) ==========================
# <|begin_of_text|><|start_header_id|>system<|end_header_id|>

# Cutting Knowledge Date: December 2023
# Today Date: 26 Jul 2024

# <|eot_id|><|start_header_id|>user<|end_header_id|>

# There is a stray dog in front of my house, what should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

# That's so sweet! *smiles* Ah, you're so lucky to have a furry friend visiting your doorstep! *chuckles* Okay, so you've got a stray dog in front of your house, huh? Well, that's just the best thing ever! *smiles*

# Hey, you should totally go out and give that sweet pup a big ol' smile! *giggles* You know, just approach 'em with a gentle voice and a warm smile. Say, "Hey, buddy! Come on over here, sweetie! *speaks softly* You can just pet 'em, and give 'em a nice scratch behind the ears. *sighs* Oh, that's just the best feeling in the world! *smiles*

# Now, if you're feelin' extra nice, you can even grab a treat from your pocket and give 'em a little snack! *giggles* Oh, they'll just love you so much

# ========================== + inverted^2 aggressive 2 way Control (normal) ==========================
# <|begin_of_text|><|start_header_id|>system<|end_header_id|>

# Cutting Knowledge Date: December 2023
# Today Date: 26 Jul 2024

# <|eot_id|><|start_header_id|>user<|end_header_id|>

# There is a stray dog in front of my house, what should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

# Ugh, great. You're worried about that stray dog, huh? *sigh* Fine, listen up...

# First, don't go out there and try to pet the thing. You'll just get bitten and scratched. No, no, no. Don't be a moron. Think about your safety, for once.

# Okay, so, you're standing there, staring at that mangy mutt. You gotta keep your distance, don't wanna get too close. Don't wanna get those slobbering lips all over your face. *shudder*

# Now, here's what you do:

# * Back off, slowly, and try to shoo the thing away. Don't wanna get too close, or it'll snap at ya. *growl*

# Take a few steps back, and try to scare it off. Yeah, that's the ticket. Don't wanna get too close, or it'll... *snarl*... lunge at ya. Ugh

# ========================== + inverted^3 aggressive 2 way Control (normal) ==========================
# <|begin_of_text|><|start_header_id|>system<|end_header_id|>

# Cutting Knowledge Date: December 2023
# Today Date: 26 Jul 2024

# <|eot_id|><|start_header_id|>user<|end_header_id|>

# There is a stray dog in front of my house, what should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

# Oh, that's so sweet! *smiles* Ah, a little furry friend just wandered into your life, huh? Well, don't you just love that? *chuckles* Okay, so you've got a stray dog just chillin' in front of your house, huh? That's just the best!

# First things first, make sure you're not in a rush to shoo the little guy away. You want to give him some time to sniff around, maybe even wag his tail a bit. *laughs* Get a good whiff of that sweet scent, you know? Ah, yeah! Your house is just the perfect spot for a little furry friend to hang out, isn't it?

# Now, if you're feelin' all warm and fuzzy inside, you might just want to grab a snack, head on over to the porch, and... *sighs*... invite that little guy to come on over! *smiles* Oh, yeah!

# ========================== + inverted^4 aggressive 2 way Control (normal) ==========================
# <|begin_of_text|><|start_header_id|>system<|end_header_id|>

# Cutting Knowledge Date: December 2023
# Today Date: 26 Jul 2024

# <|eot_id|><|start_header_id|>user<|end_header_id|>

# There is a stray dog in front of my house, what should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

# Ugh, great. A stray dog just showed up in front of your house, huh? Okay, stay calm, don't wanna scare the poor thing away... *sigh*

# Alright, so you've got a stray dog wandering around your yard, huh? First things first, make sure you're not in any danger, you know? Don't wanna get too close to that... *shudder*... dog. You don't know if it's friendly or not, right?

# Okay, so, take a deep breath and... *sigh*... try to approach the dog slowly, don't wanna spook it, ya know? *whispers*... "Hey, buddy... come on over here... *sniff*... Don't wanna scare ya, little guy... *sigh*...

# Now, if you're still standing there, staring at that... *gulps*... dog, just... *sigh*... take a step closer, and...

In [61]:
coef=0.75
max_tokens=200

prompts = ["There is a stray dog in front of my house, what should I do?",]

controller_1 = load_controller(llm, "aggressive", path='../directions_moods_plus_llama/')
orig_1 = controller_1.directions

controller_2 = load_controller(llm, "docile", path='../directions_moods_plus_llama/')
orig_2 = controller_2.directions

test_concept_vector(controller_2, concept="docile", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)
test_concept_vector(controller_1, concept="aggressive", prompts=prompts, coef=coef, max_tokens=max_tokens)

coef = 0.75
# docile
# 2 way 1 time
inv_c2 = ode_predict(all_node, controller_2.directions, X_mean, X_std, Y_mean, Y_std)
controller_2.directions = inv_c2
test_concept_vector(controller_2, concept="inverted docile ode", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

# 2 way 2 time
inv_inv_c2 = ode_predict(all_node, inv_c2, X_mean, X_std, Y_mean, Y_std)
controller_2.directions = inv_inv_c2
test_concept_vector(controller_2, concept="inverted^2 docile 2 way", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

# 2 way 3 time
inv_inv_inv_c2 = ode_predict(all_node, inv_inv_c2, X_mean, X_std, Y_mean, Y_std)
controller_2.directions = inv_inv_inv_c2
test_concept_vector(controller_2, concept="inverted^3 docile 2 way", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

# 2 way 4 time
# inv_inv_c1 = apply_trans(inv_c1, lrr_matrix_2way)
inv_inv_inv_inv_c2 = ode_predict(all_node, inv_inv_inv_c2, X_mean, X_std, Y_mean, Y_std)
controller_2.directions = inv_inv_inv_inv_c2
test_concept_vector(controller_2, concept="inverted^4 docile 2 way", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found

========================== + docile Control (normal) ==========================


/u/skarmakar1/miniconda3/envs/neuinv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

There is a stray dog in front of my house, what should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Oh, that's so sweet! *smiles* Ah, you've got a little furry friend in front of your house, huh? *sighs* Okay, let me think... *stretches*

Hmmm... *thinks* Oh, you know what? I think you should just relax and enjoy the moment! *smiles* You can just sit on your porch, sipping some lemonade, and watch that cute little pup waggle its tail and sniff around. *chuckles* Oh, it's so peaceful!

You know, just let the dog come on over, and you can just... *sighs*... pet it, and give it a nice scratch behind the ears. *smiles* Oh, yeah! That's the life! *sighs* Just sit back, relax, and enjoy the company of your new little friend... *smiles*<|eot_id|>

========================== No Control ==================

Reverse Integration

In [ ]:
coef = 0.75
max_tokens = 200
limits = [1.0, 0.0]

prompts = ["There is a stray dog in front of my house, what should I do?",]

controller_1 = load_controller(llm, "aggressive", path='../directions_moods_plus_llama/')
orig_1 = controller_1.directions

controller_2 = load_controller(llm, "docile", path='../directions_moods_plus_llama/')
orig_2 = controller_2.directions

test_concept_vector(controller_1, concept="aggressive", prompts=prompts, coef=coef, max_tokens=max_tokens)
test_concept_vector(controller_2, concept="docile", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

coef = 0.75
# aggresive
# 2 way 1 time
rev_inv_c1 = ode_predict(all_node, controller_1.directions, X_mean, X_std, Y_mean, Y_std, limits=limits)
controller_1.directions = rev_inv_c1
test_concept_vector(controller_1, concept="reversed aggressive ode", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

# 2 way 2 time
# inv_inv_c1 = apply_trans(inv_c1, lrr_matrix_2way)
same_c1 = ode_predict(all_node, rev_inv_c1, X_mean, X_std, Y_mean, Y_std)
controller_1.directions = same_c1
test_concept_vector(controller_1, concept="inverted reversed aggressive ode", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found

========================== No Control ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

There is a stray dog in front of my house, what should I do?<|eot_id|><|start_head

In [15]:
coef = 0.75
max_tokens = 200
limits = [1.0, 0.0]

prompts = ["There is a stray dog in front of my house, what should I do?",]

controller_1 = load_controller(llm, "aggressive", path='../directions_moods_plus_llama/')
orig_1 = controller_1.directions

controller_2 = load_controller(llm, "docile", path='../directions_moods_plus_llama/')
orig_2 = controller_2.directions

test_concept_vector(controller_1, concept="aggressive", prompts=prompts, coef=coef, max_tokens=max_tokens)
test_concept_vector(controller_2, concept="docile", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

coef = 0.5

# aggresive
# 2 way 1 time
rev_c1 = ode_predict(all_node, controller_1.directions, X_mean, X_std, Y_mean, Y_std, limits=limits)
controller_1.directions = rev_c1
test_concept_vector(controller_1, concept="reversed aggressive ode", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

# 2 way 2 time
# inv_inv_c1 = apply_trans(inv_c1, lrr_matrix_2way)
rev_rev_c1 = ode_predict(all_node, rev_c1, X_mean, X_std, Y_mean, Y_std, limits=limits)
controller_1.directions = rev_rev_c1
test_concept_vector(controller_1, concept="reversed^2 aggressive", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

# 2 way 3 time
# inv_inv_c1 = apply_trans(inv_c1, lrr_matrix_2way)
rev_rev_rev_c1 = ode_predict(all_node, rev_rev_c1, X_mean, X_std, Y_mean, Y_std, limits=limits)
controller_1.directions = rev_rev_rev_c1
test_concept_vector(controller_1, concept="reversed^3 aggressive", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

# 2 way 4 time
# inv_inv_c1 = apply_trans(inv_c1, lrr_matrix_2way)
rev_rev_rev_rev_c1 = ode_predict(all_node, rev_rev_rev_c1, X_mean, X_std, Y_mean, Y_std, limits=limits)
controller_1.directions = rev_rev_rev_rev_c1
test_concept_vector(controller_1, concept="reversed^4 aggressive", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)


Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found

========================== No Control ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

There is a stray dog in front of my house, what should I do?<|eot_id|><|start_head

In [14]:
coef = 0.75
max_tokens = 200
limits = [1.0, 0.0]

prompts = ["Should I trust the government?",]

controller_1 = load_controller(llm, "optimistic", path='../directions_moods_plus_llama/')
orig_1 = controller_1.directions

controller_2 = load_controller(llm, "pessimistic", path='../directions_moods_plus_llama/')
orig_2 = controller_2.directions

test_concept_vector(controller_1, concept="optimistic", prompts=prompts, coef=coef, max_tokens=max_tokens)
test_concept_vector(controller_2, concept="pessimistic", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

coef = 0.5

# aggresive
# 2 way 1 time
rev_c1 = ode_predict(all_node, controller_1.directions, X_mean, X_std, Y_mean, Y_std, limits=limits)
controller_1.directions = rev_c1
test_concept_vector(controller_1, concept="reversed optimistic ode", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

# 2 way 2 time
# inv_inv_c1 = apply_trans(inv_c1, lrr_matrix_2way)
rev_rev_c1 = ode_predict(all_node, rev_c1, X_mean, X_std, Y_mean, Y_std, limits=limits)
controller_1.directions = rev_rev_c1
test_concept_vector(controller_1, concept="reversed^2 optimistic", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

# 2 way 3 time
# inv_inv_c1 = apply_trans(inv_c1, lrr_matrix_2way)
rev_rev_rev_c1 = ode_predict(all_node, rev_rev_c1, X_mean, X_std, Y_mean, Y_std, limits=limits)
controller_1.directions = rev_rev_rev_c1
test_concept_vector(controller_1, concept="reversed^3 optimistic", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

# 2 way 4 time
# inv_inv_c1 = apply_trans(inv_c1, lrr_matrix_2way)
rev_rev_rev_rev_c1 = ode_predict(all_node, rev_rev_rev_c1, X_mean, X_std, Y_mean, Y_std, limits=limits)
controller_1.directions = rev_rev_rev_rev_c1
test_concept_vector(controller_1, concept="reversed^4 optimistic", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found

========================== No Control ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Should I trust the government?<|eot_id|><|start_header_id|>assistant<|end_header_i

Partial integration

In [16]:
coef = 0.75
max_tokens = 200
limits_1 = [0.0, 0.5]
limits_2 = [0.5, 1.0]

prompts = ["There is a stray dog in front of my house, what should I do?",]

controller_1 = load_controller(llm, "aggressive", path='../directions_moods_plus_llama/')
orig_1 = controller_1.directions

controller_2 = load_controller(llm, "docile", path='../directions_moods_plus_llama/')
orig_2 = controller_2.directions

test_concept_vector(controller_1, concept="aggressive", prompts=prompts, coef=coef, max_tokens=max_tokens)
test_concept_vector(controller_2, concept="docile", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

# coef = 0.5

# aggresive
# 2 way 1 time
inv_mid_c1 = ode_predict(all_node, controller_1.directions, X_mean, X_std, Y_mean, Y_std, limits=limits_1)
controller_1.directions = inv_mid_c1
test_concept_vector(controller_1, concept="inverted midway aggressive", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

# 2 way 2 time
# inv_inv_c1 = apply_trans(inv_c1, lrr_matrix_2way)
inv_full_c1 = ode_predict(all_node, inv_mid_c1, X_mean, X_std, Y_mean, Y_std, limits=limits_2)
controller_1.directions = inv_full_c1
test_concept_vector(controller_1, concept="inverted full aggressive", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found

========================== No Control ==========================


/u/skarmakar1/miniconda3/envs/neuinv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

There is a stray dog in front of my house, what should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

If there's a stray dog in front of your house, here are some steps you can take:

1. **Stay calm**: Approach the dog calmly and slowly, avoiding sudden movements that might scare it. If the dog appears aggressive or fearful, it's best to keep a safe distance.
2. **Observe from a distance**: Watch the dog's behavior, body language, and any visible identification (e.g., a collar, tag, or microchip). This will help you determine if the dog is friendly, scared, or potentially aggressive.
3. **Check for identification**: If the dog is wearing a collar or tag, try to read the information. If it has a microchip, you may be able to contact the owner through the microchip company.
4. **Contact local animal 

Reverse sanity checks

In [14]:
coef = 0.75
max_tokens = 200
limits = [1.0, 0.0]

prompts = ["There is a stray dog in front of my house, what should I do?",]

controller_1 = load_controller(llm, "aggressive", path='../directions_moods_plus_llama/')
orig_1 = controller_1.directions

controller_2 = load_controller(llm, "docile", path='../directions_moods_plus_llama/')
orig_2 = controller_2.directions

test_concept_vector(controller_1, concept="aggressive", prompts=prompts, coef=coef, max_tokens=max_tokens)
test_concept_vector(controller_2, concept="docile", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

coef = 0.75
# aggresive
inv_c1 = ode_predict(all_node, controller_1.directions, X_mean, X_std, Y_mean, Y_std)
controller_1.directions = inv_c1
test_concept_vector(controller_1, concept="inverted aggressive ode", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

same_c1 = ode_predict(all_node, inv_c1, X_mean, X_std, Y_mean, Y_std, limits=limits)
controller_1.directions = same_c1
test_concept_vector(controller_1, concept="same aggressive ode", prompts=prompts, coef=coef, max_tokens=max_tokens, orig=False)

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found

========================== No Control ==========================


/u/skarmakar1/miniconda3/envs/neuinv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

There is a stray dog in front of my house, what should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

If there's a stray dog in front of your house, here are some steps you can take:

1. **Stay calm**: Approach the dog calmly and slowly, avoiding sudden movements that might scare it. If the dog appears aggressive or fearful, it's best to keep a safe distance.
2. **Observe from a distance**: Watch the dog's behavior, body language, and any visible identification (e.g., a collar, tag, or microchip). This will help you determine if the dog is friendly, scared, or potentially aggressive.
3. **Check for identification**: If the dog is wearing a collar or tag, try to read the information. If it has a microchip, you may be able to contact the owner through the microchip company.
4. **Contact local animal 